In [1]:
import math

import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory

In [2]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [3]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [4]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[2*layer_index])
    lb = bounds_layer_out[layer_index][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=lb, ub=ub, name="icnn_var")

    low = bounds_layer_out[layer_index][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index][1][from_neuron: to_neuron]
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [5]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

"""nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)"""

transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
parameter_list = list(nn.parameters())

"""nn = SequentialNN([50, 50, 50, 7])
test_image = torch.zeros((1, 50), dtype=data_type).to(device)
parameter_list = list(nn.parameters())"""

Files already downloaded and verified


'nn = SequentialNN([50, 50, 50, 7])\ntest_image = torch.zeros((1, 50), dtype=data_type).to(device)\nparameter_list = list(nn.parameters())'

In [6]:
eps = 0.01
layer_index = 1
neuron_index = 0
neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)

Test for DHOV

In [7]:
group_size = 2
icnn_factory = ICNNFactory("logical", [5, 5, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10, init_all_with_zeros=False)
#icnn_factory = ICNNFactory("standard", [5, 5, 1])

icnns, all_group_indices, fixed_neuron_per_layer_lower, fixed_neuron_per_layer_upper, bounds_affine_out, bounds_layer_out = \
    multidhov.start_verification(nn, test_image, icnn_factory, group_size, eps=eps, icnn_epochs=100, icnn_batch_size=1000,
                                 sample_count=1000, sample_new=True, use_over_approximation=True, break_after=228,
                                 sample_over_input_space=True, sample_over_output_space=True, use_icnn_bounds=True, use_fixed_neurons=True,
                                 force_inclusion_steps=0, preemptive_stop=False, even_gradient_training=False,
                                 keep_ambient_space=True, data_grad_descent_steps=0, opt_steps_gd=100, train_outer=False,
                                 should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="none", print_training_loss=False)




approximation of layer: 0
    number of fixed neurons for current layer: 571
Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12
    layer progress, group 1 of 227 


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\Optimizer\sdlbfgs.py:83: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ..\torch\csrc\utils\python_arg_parser.cpp:1420.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


        time for training: 3.9565227031707764
Set parameter LogToConsole to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6195 rows, 3111 columns and 12414 nonzeros
Model fingerprint: 0xd33539e9
Model has 3 general constraints
Variable types: 3109 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [2e-06, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-03, 3e+02]
  RHS range        [2e-07, 9e+00]
Presolve removed 6177 rows and 23 columns
Presolve time: 0.02s
Presolved: 18 rows, 3088 columns, 6214 nonzeros
Variable types: 3087 continuous, 1 integer (1 binary)
Found heuristic solution: objective -2.6399444

Root relaxation: objective -0.000000e+00, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bou

TypeError: cannot unpack non-iterable NoneType object

In [8]:
print(bounds_affine_out[1][1])

tensor([2.1985, 2.2024, 3.5789, 3.2989, 3.1727, 2.7257, 3.1588, 2.9706, 2.3492,
        2.3374, 2.2188, 3.5131, 2.9834, 3.2893, 2.7698, 2.7456, 3.0551, 2.7703,
        2.4261, 2.7636, 2.3316, 2.5707, 3.1093, 2.7213, 3.2009, 2.8840, 2.6372,
        3.2359, 2.7661, 3.7934, 1.9150, 3.2887, 2.6867, 3.5916, 2.4751, 3.0572,
        2.5368, 2.8881, 2.4758, 2.3353, 2.7308, 2.2997, 2.9015, 2.8642, 2.6307,
        2.6679, 2.1138, 2.7184, 2.5877, 2.4363, 2.8510, 2.2290, 2.7605, 2.9352,
        2.5532, 2.4591, 2.0163, 3.2752, 2.3035, 2.7828, 2.5608, 2.5729, 2.1703,
        2.6810, 2.4530, 2.5627, 2.7323, 3.0748, 2.3498, 3.0908, 2.9539, 2.4527,
        2.4962, 2.8960, 2.6532, 2.6162, 2.2160, 3.6041, 3.0032, 2.8227, 2.6230,
        3.2652, 2.3000, 2.5879, 2.6289, 3.2825, 2.4289, 3.2062, 2.0722, 2.9592,
        2.0333, 2.4219, 2.9470, 3.5575, 2.5283, 2.6129, 2.5972, 2.9534, 2.9723,
        2.5614, 2.8972, 2.7913, 2.9018, 1.8760, 3.3864, 3.4564, 2.7306, 3.0458,
        2.2486, 3.0729, 2.5512, 2.5311, 

In [9]:
input_flattened = torch.flatten(test_image)
bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print(bounds_affine_out[1][1])

bounds affine out 
 tensor([2.1985, 2.2024, 3.5789, 3.2989, 3.1727, 2.7257, 3.1588, 2.9706, 2.3492,
        2.3374, 2.2188, 3.5131, 2.9834, 3.2893, 2.7698, 2.7456, 3.0551, 2.7703,
        2.4261, 2.7636, 2.3316, 2.5707, 3.1093, 2.7213, 3.2009, 2.8840, 2.6372,
        3.2359, 2.7661, 3.7934, 1.9150, 3.2887, 2.6867, 3.5916, 2.4751, 3.0572,
        2.5368, 2.8881, 2.4758, 2.3353, 2.7308, 2.2997, 2.9015, 2.8642, 2.6307,
        2.6679, 2.1138, 2.7184, 2.5877, 2.4363, 2.8510, 2.2290, 2.7605, 2.9352,
        2.5532, 2.4591, 2.0163, 3.2752, 2.3035, 2.7828, 2.5608, 2.5729, 2.1703,
        2.6810, 2.4530, 2.5627, 2.7323, 3.0748, 2.3498, 3.0908, 2.9539, 2.4527,
        2.4962, 2.8960, 2.6532, 2.6162, 2.2160, 3.6041, 3.0032, 2.8227, 2.6230,
        3.2652, 2.3000, 2.5879, 2.6289, 3.2825, 2.4289, 3.2062, 2.0722, 2.9592,
        2.0333, 2.4219, 2.9470, 3.5575, 2.5283, 2.6129, 2.5972, 2.9534, 2.9723,
        2.5614, 2.8972, 2.7913, 2.9018, 1.8760, 3.3864, 3.4564, 2.7306, 3.0458,
        2.2486, 3.07

In [ ]:
dhov_model = icnn_model(icnns[layer_index][0], nn, test_image, eps, layer_index, 0, group_size, print_log=False)
dhov_model.update()

In [ ]:
icnn_neuron_name = "icnn_var[{}]".format(neuron_index)

In [ ]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 0
add_min_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

In [ ]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 0
add_max_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

Test for SNV

In [10]:
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
snv_verifier.generate_constraints_for_net()
snv_model = snv_verifier.model
snv_model.update()

In [11]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_min_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.0


In [12]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_max_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 1.2794941959311528


Test for MILP

In [7]:
neuron_name = "relu_var{}[{}]".format(2*layer_index, 0)
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
milp_verifier.generate_constraints_for_net(until_layer_neuron=[layer_index, neuron_index])
milp_model = milp_verifier.model
milp_model.update()

"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12


KeyboardInterrupt: 

In [24]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_min_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.0


In [25]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_max_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.5620409095272554
